### Developement script for combining autosegmentation with RADIPOP 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# imports 
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'
os.environ['MKL_SERVICE_FORCE_INTEL'] = '1'
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import SimpleITK as sitk

import nibabel as nib
from tqdm import tqdm

import totalsegmentator
import totalsegmentator.python_api



import radipop_utils 
import radipop_utils.visualization
import radipop_utils.features
import radipop_utils.utils
import radipop_utils.data
import radipop_utils.inference

from radipop_utils.features import (
    convert_and_extract_from_nii,
)
import sklearn
from sklearn.preprocessing  import StandardScaler

# load user/ system specific env variables:
from dotenv import dotenv_values, find_dotenv
config = dotenv_values(find_dotenv())  # load environment variables as dictionary

# path to the data. You will (likely need to change this)
DATA_ROOT_DIRECTORY = Path(config["DATA_ROOT_DIRECTORY"])


# needed for loading the settings for the radiomics feature extractor 
path = Path(os.path.abspath(radipop_utils.__file__))
RADIPOP_PACKAGE_ROOT = path.parent.parent

# These all need to match with the loaded model
DATASET = "Dataset125_LSS"
RADIOMICS_OPTION = "radipop_222"
fe_settings_path = RADIPOP_PACKAGE_ROOT / "yaml" / "radiomics_fe_setttings_CT_no_preprocessing_spacing_222.yaml"
window_location_middle = 50
window_width = 500


# dev flags:
# CONVERT_DCM2NII = False
# RUN_SEGMENTATION = False
# RUN_FEATURE_EXTRACTION = False
# SAVE_INTERMEDIATE_RESULTS = False

CONVERT_DCM2NII = True
RUN_SEGMENTATION = True
RUN_FEATURE_EXTRACTION = True
SAVE_INTERMEDIATE_RESULTS = True

In [ ]:
# convert from dictom to nifti
sample_dicom_path = Path("/home/cwatzenboeck/data/project_segmenation_ls/2024_muw_additional_DICOMs/V.313 horos")
tmp_wd_path  = Path("/home/cwatzenboeck/tmp/radipop_tmp_wd")
subfolder_name = "patient_dev"

if CONVERT_DCM2NII:
    radipop_utils.utils.dcm2nii(sample_dicom_path, tmp_wd_path, out_id = subfolder_name, verbose=True)

In [ ]:
# auto segmentation
input_path = tmp_wd_path / subfolder_name / "base.nii.gz"
output_path = tmp_wd_path / subfolder_name / "mask_ts.nii.gz"

# DEV: load patient patient0027 instead to compare
# input_path = Path("/home/cwatzenboeck/data/cirdata/nnUNet_raw/Dataset125_LSS/imagesAll/patient0027_0000.nii.gz")

if __name__ == "__main__":
    if RUN_SEGMENTATION:
        print("Loading volume: ", input_path)
        input_img = nib.load(input_path)
        output_img = totalsegmentator.python_api.totalsegmentator(input_img)
        nib.save(output_img, output_path)
        print("Saved mask to: ", output_path)
    
    

In [ ]:
# # load image and plot overlayed masks + image for one slice:
show_plot = True
image_loc = input_path
mask_loc = output_path
plt.close('all')
if show_plot:
    idx = 260
    mask = sitk.ReadImage(mask_loc)
    img = convert_and_extract_from_nii(image_loc, out_range=[0, 1.0], wl=50, ww=500, dtype=np.float64)  

    img_array = sitk.GetArrayFromImage(img)
    mask_array = sitk.GetArrayFromImage(mask)
    image_xy_slice = img_array[idx, :, :]
    label_xy_slice = mask_array[idx, :, :]

    crop = lambda x: radipop_utils.visualization.crop_image(x, top_ratio=0.15, bottom_ratio=0.2, left_ratio=0.05, right_ratio=0.02)
    image_xy_slice = crop(image_xy_slice)
    label_xy_slice = crop(label_xy_slice)

    color_dictionary = {
        'violet': (170/255, 48/255, 127/255, 0.35),
        'blue':   (0, 0, 1, 0.35),
    }

    # Overlay masks on the image slice
    fig = radipop_utils.visualization.overlay_masks_and_show_bare_image_next_to_it(
        image_xy_slice,
        [label_xy_slice == 5, label_xy_slice == 1],
        [color_dictionary['violet'], color_dictionary['blue']],
        facecolor='white'
    )

    # Display the combined figure
    display(fig)

In [ ]:
# extract radiomics features

image_loc = input_path
mask_loc = output_path

tissue_class_dct = {"liver": 5, "spleen": 1}

mask = sitk.ReadImage(output_path)
img = convert_and_extract_from_nii(image_loc, out_range=[0, 1.0], wl=window_location_middle, ww=window_width, dtype=np.float64)  

radiomics_dataframes = {}   
if RUN_FEATURE_EXTRACTION: 
    for tissue_type in tqdm(tissue_class_dct.keys()):
        print(f"Extracting features for tissue type: {tissue_type}")
        features_df = radipop_utils.features.extract_radiomics_features(img, mask == tissue_class_dct[tissue_type], fe_settings_path)
        radiomics_dataframes[tissue_type] = features_df
    if SAVE_INTERMEDIATE_RESULTS:
        for tissue_type, features_df in radiomics_dataframes.items():
            out_file =tmp_wd_path / subfolder_name  / f"Features_{tissue_type}.xlsx"
            features_df.to_excel(out_file)
            print(f"Saved features to: {out_file}")
else: 
    for tissue_type in tissue_class_dct.keys():
        src = tmp_wd_path / subfolder_name  / f"Features_{tissue_type}.xlsx"
        radiomics_dataframes[tissue_type] = pd.read_excel(src)            
        print(f"Loaded radiomics features for tissue type: {tissue_type} from: {src}")
        
dfc = radipop_utils.data.combined_radiomics_features(radiomics_dataframes)

In [ ]:
# load_models_and_params
# TODO make platform independent
model_dir = DATA_ROOT_DIRECTORY / "radiomics" / DATASET / "regression" / RADIOMICS_OPTION
loaded_models, loaded_params, models_bare = radipop_utils.inference.load_models_and_params(model_dir = model_dir)

In [ ]:
# load normalization_df and scaler
normalization_df = radipop_utils.data.load_normalization_df(model_dir)
scaler = radipop_utils.data.make_scaler_from_normalization_df(normalization_df)
X = scaler.transform(dfc)


# predict
loaded_models["RF"].predict(X)

In [ ]:
# # Compare with different mask -> different radiomoics features
# df_Tr, df_iTs, df_eTs = radipop_utils.data.quickload_or_combine_radiomics_data(
#     DATASET=DATASET,
#     RADIOMICS_OPTION=RADIOMICS_OPTION,
#     DATA_ROOT_DIRECTORY=DATA_ROOT_DIRECTORY,
#     verbose=True
# )
# X_Tr, Y_Tr, X_iTs, Y_iTs, X_eTs, Y_eTs = radipop_utils.data.preprocess_data(df_Tr, df_iTs, df_eTs, normalize_X=True)
# loaded_models["RF"].predict(X_Tr[:1,:])

